In [1]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tqdm import tqdm
 
import nltk
#en_stop = set(nltk.corpus.<a href="https://thinkinfi.com/how-to-remove-stop-words-in-python/" data-internallinksmanager029f6b8e52c="5" title="How To Remove Stop words In Python" target="_blank" rel="noopener">stopwords</a>.words('english'))
 
from gensim.models import FastText
 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
 
# <a href="https://thinkinfi.com/difference-between-stemming-and-lemmatizing-and-where-to-use/" data-internallinksmanager029f6b8e52c="7" title="Difference between stemming and lemmatizing and where to use" target="_blank" rel="noopener">Lemmatization</a>
from nltk import WordNetLemmatizer
stemmer = WordNetLemmatizer()

In [2]:
train_text =np.load('Mal_train.npy')
print(train_text.shape)
train_text = train_text.tolist()
#print(train_text)
train_label =np.load('Mal_train_label.npy')
print(train_label.shape)
train_label = train_label.tolist()
#print(train_label)

(1000,)
(1000,)


In [3]:
valid_text =np.load('Mal_val.npy')
print(valid_text.shape)
valid_text = valid_text.tolist()
#print(valid_text)
valid_label =np.load('Mal_val_label.npy')
print(valid_label.shape)
valid_label = valid_label.tolist()
#print(valid_label)

(100,)
(100,)


In [4]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

Some layers from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

In [6]:
input_ids = tokenizer(train_text, return_tensors='tf',truncation=True,padding=True)['input_ids']

#input_ids = tokenizer(valid_text, return_tensors='tf',truncation=True,padding=True)['input_ids']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
import math

In [8]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(input_ids)/Batch_size)

In [9]:
import time

In [10]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(input_ids)
        out = model(input_ids[int(start):int(Endu)])
    else:
        out = model(input_ids[int(start):int(Endu)])
    if i == 0:
        total = out.pooler_output 
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out.pooler_output],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [11]:
len(total)

1000

In [12]:
total

<tf.Tensor: shape=(1000, 768), dtype=float32, numpy=
array([[ 0.05744659,  0.8334571 , -0.4579291 , ..., -0.23478337,
        -0.31508487, -0.21995133],
       [ 0.02108973,  0.8143126 , -0.46389127, ..., -0.2219873 ,
        -0.3542156 , -0.24277815],
       [ 0.21211018,  0.8828258 , -0.35285172, ..., -0.25310165,
        -0.17590131, -0.15937829],
       ...,
       [ 0.08203451,  0.8455281 , -0.44398186, ..., -0.23515292,
        -0.29266575, -0.20290166],
       [ 0.25017798,  0.8933832 , -0.31349182, ..., -0.25508717,
        -0.12339611, -0.14075027],
       [ 0.11087467,  0.856152  , -0.43033743, ..., -0.23592502,
        -0.27159697, -0.19086742]], dtype=float32)>

In [13]:
list(out)

['last_hidden_state', 'pooler_output']

In [14]:
#out['last_hidden_state']

In [15]:
# input_ids.

In [16]:
!nvidia-smi

Sat Oct 23 10:02:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   58C    P0    38W /  N/A |   6112MiB /  6144MiB |     84%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
X_train = np.array(total)
np.save('Indice_BERT_X_train.npy', X_train)

In [18]:
from transformers import TFAutoModel
model = TFAutoModel.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

Some layers from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at neuralspace-reverie/indic-transformers-hi-bert.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [19]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('neuralspace-reverie/indic-transformers-hi-bert')

In [20]:
input_ids = tokenizer(valid_text, return_tensors='tf',truncation=True,padding=True)['input_ids']

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [21]:
Batch_size = 64
start = 0;Endu = Batch_size
fullsize = math.floor(len(input_ids)/Batch_size)

In [22]:
for i in range(fullsize):
    if i+1 == fullsize:
        Endu = len(input_ids)
        out = model(input_ids[int(start):int(Endu)])
    else:
        out = model(input_ids[int(start):int(Endu)])
    if i == 0:
        total = out.pooler_output 
    else:
        #torch.cat((total,out.last_hidden_state),0)
        total = tf.concat([total, out.pooler_output],0)
    start = Batch_size * (i+1)
    Endu = Batch_size * (i+2)
    print(i)

0


In [23]:
X_valid = np.array(total)
np.save('Indice_BERT_X_val.npy', X_valid)

In [24]:
# Y_train = np.array(train_label)
# np.save('Indice_BERT_Y_train.npy',Y_train)

# Y_val = np.array(valid_label)
# np.save('Indice_BERT_Y_val.npy',Y_val)

In [25]:
X_train = np.load('Indice_BERT_X_train.npy')
X_val = np.load('Indice_BERT_X_val.npy')

In [26]:
len(X_train)

1000

In [27]:
len(train_label)

1000

In [28]:
Y_train = np.array(train_label)
Y_val = np.array(valid_label)

In [29]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\OneDrive - Amrita Vishwa Vidyapeetham\Amrita-Mtech-CEN\ProjectWork\MTech Thesis\Denoise in Code mix data\Experiments\Malyalam\Indice_BERT\1000 and 100 dataset

23-10-2021  10:01    <DIR>          .
23-10-2021  09:49    <DIR>          ..
23-10-2021  09:58    <DIR>          .ipynb_checkpoints
23-10-2021  10:01            34,018 Indice_BERT.ipynb
23-10-2021  09:58            33,936 Indice_BERT_Denoised_SVM_DT_RF.ipynb
23-10-2021  10:02         3,072,128 Indice_BERT_X_train.npy
23-10-2021  10:03           307,328 Indice_BERT_X_val.npy
23-10-2021  09:17         6,268,128 Mal_train.npy
23-10-2021  09:17             8,128 Mal_train_label.npy
23-10-2021  09:17            84,128 Mal_val.npy
23-10-2021  09:17               928 Mal_val_label.npy
22-10-2021  13:30               360 WR_LS_Denoising.py
               9 File(s)      9,809,082 bytes
               3 Dir(s)  331,885,748,224 bytes free


# SVM

In [30]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

In [31]:
svc = SVC(decision_function_shape='ovo')
svc.fit(X_train,Y_train)

SVC(decision_function_shape='ovo')

In [32]:
pred_svc = svc.predict(X_val)

In [33]:
confusion_matrix(Y_val,pred_svc)

array([[ 0, 19,  0,  0,  1],
       [ 0, 18,  0,  1,  1],
       [ 1, 18,  0,  0,  1],
       [ 1, 18,  0,  1,  0],
       [ 0, 20,  0,  0,  0]], dtype=int64)

In [34]:
print(classification_report(Y_val,pred_svc))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.19      0.90      0.32        20
           2       0.00      0.00      0.00        20
           3       0.50      0.05      0.09        20
           4       0.00      0.00      0.00        20

    accuracy                           0.19       100
   macro avg       0.14      0.19      0.08       100
weighted avg       0.14      0.19      0.08       100



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## DecisionTree

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
Dtree = DecisionTreeClassifier(criterion='entropy', random_state=10)

In [37]:
Dtree.fit(X_train,Y_train)

DecisionTreeClassifier(criterion='entropy', random_state=10)

In [38]:
pred_Dtree = Dtree.predict(X_val)

In [39]:
confusion_matrix(Y_val,pred_Dtree)

array([[ 0,  1,  0, 19,  0],
       [ 0,  0,  2, 18,  0],
       [ 0,  0,  3, 17,  0],
       [ 0,  0,  3, 17,  0],
       [ 0,  0,  1, 19,  0]], dtype=int64)

In [40]:
print(classification_report(Y_val,pred_Dtree))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.00      0.00      0.00        20
           2       0.33      0.15      0.21        20
           3       0.19      0.85      0.31        20
           4       0.00      0.00      0.00        20

    accuracy                           0.20       100
   macro avg       0.10      0.20      0.10       100
weighted avg       0.10      0.20      0.10       100



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

In [42]:
RFC = RandomForestClassifier()
RFC.fit(X_train,Y_train)

RandomForestClassifier()

In [43]:
pred_RFC = RFC.predict(X_val)

In [44]:
confusion_matrix(Y_val,pred_RFC)

array([[ 0,  8,  0,  0, 12],
       [ 0, 13,  0,  1,  6],
       [ 0,  7,  0,  1, 12],
       [ 0, 10,  0,  1,  9],
       [ 0,  8,  0,  0, 12]], dtype=int64)

In [45]:
print(classification_report(Y_val,pred_RFC))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.28      0.65      0.39        20
           2       0.00      0.00      0.00        20
           3       0.33      0.05      0.09        20
           4       0.24      0.60      0.34        20

    accuracy                           0.26       100
   macro avg       0.17      0.26      0.16       100
weighted avg       0.17      0.26      0.16       100



C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\phvpa\anaconda3\envs\MuRIL\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
